In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce

%matplotlib inline
pd.options.mode.chained_assignment = None

SEED = 45

/Users/alirezafaghaninia/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
test = pd.read_csv('data/test.csv', usecols=['item_id'])
train = pd.read_csv('data/train.csv', usecols=['item_id', 'deal_probability'])

In [3]:
periods = pd.read_csv('predictions/periods.csv').rename(columns={'deal_probability': 'periods_predicted'})
periods_train = pd.read_csv('predictions/periods_train.csv').rename(columns={'deal_probability': 'periods_predicted'})

In [4]:
activation_date = pd.read_csv('predictions/activation_date.csv').rename(columns={'deal_probability': 'activation_predicted'})
activation_date_train = pd.read_csv('predictions/activation_date_train.csv').rename(columns={'deal_probability': 'activation_predicted'})

In [5]:
tabular = pd.read_csv('predictions/tabular.csv').rename(columns={'deal_probability': 'tabular_predicted'})
tabular_train = pd.read_csv('predictions/tabular_train.csv').rename(columns={'deal_probability': 'tabular_predicted'})

In [6]:
nlp = pd.read_csv('predictions/nlp.csv').rename(columns={'deal_probability': 'nlp_predicted'})
nlp_train = pd.read_csv('predictions/nlp_train.csv').rename(columns={'deal_probability': 'nlp_predicted'})

## Training the ensembling model

In [7]:
train = reduce(lambda left,right: pd.merge(left,right,on='item_id', how='left'), 
              [train, periods_train, activation_date_train, tabular_train, nlp_train]).set_index('item_id')

In [8]:
train.head()

,deal_probability,periods_predicted,activation_predicted,tabular_predicted,nlp_predicted
item_id,,,,,
b912c3c6a6ad,0.12789,0.152303,0.139307,0.201496,0.109393
2dac0150717d,0.00000,0.127730,0.138812,0.296981,0.173961
ba83aefab5dc,0.43177,0.016474,0.138020,0.240076,0.256733
02996f1dd2ea,0.80323,0.232459,0.141566,0.232893,0.196804
7c90be56d2ab,0.20797,0.125294,0.141675,0.388763,0.344846


In [9]:
test = reduce(lambda left,right: pd.merge(left,right,on='item_id', how='left'), 
              [test, periods, activation_date, tabular, nlp]).set_index('item_id')

In [10]:
test.head()

,periods_predicted,activation_predicted,tabular_predicted,nlp_predicted
item_id,,,,
6544e41a8817,0.073750,0.572439,0.056549,0.055330
65b9484d670f,0.289427,0.476374,0.138432,0.113738
8bab230b2ecd,0.153612,0.346118,0.190808,0.170690
8e348601fefc,0.506213,0.346118,0.246890,0.314884
8bd2fe400b89,0.115146,0.540526,0.230973,0.225980


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.drop('deal_probability', axis=1), 
                                                    train['deal_probability'], 
                                                    random_state=SEED)

/Users/alirezafaghaninia/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/alirezafaghaninia/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from utils import config_dict
model = RandomForestRegressor(n_estimators=20, random_state=SEED)
# model = LinearRegression(n_jobs=-1)

/Users/alirezafaghaninia/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/alirezafaghaninia/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/alirezafaghaninia/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [13]:
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
           oob_score=False, random_state=45, verbose=0, warm_start=False)

In [14]:
model.score(X_test, y_test)

0.17669922043084596

In [15]:
rmse = np.sqrt(np.mean((model.predict(X_test) - y_test) ** 2))
rmse

0.23590012884628098

In [16]:
test.head()

,periods_predicted,activation_predicted,tabular_predicted,nlp_predicted
item_id,,,,
6544e41a8817,0.073750,0.572439,0.056549,0.055330
65b9484d670f,0.289427,0.476374,0.138432,0.113738
8bab230b2ecd,0.153612,0.346118,0.190808,0.170690
8e348601fefc,0.506213,0.346118,0.246890,0.314884
8bd2fe400b89,0.115146,0.540526,0.230973,0.225980


In [17]:
len(tabular_train)

1503424

In [18]:
len(periods_train)

1503424

In [19]:
test['deal_probability'] = model.predict(test.values)

In [20]:
test['deal_probability'][test['deal_probability'] < 0.0] = 0.0
test['deal_probability'][test['deal_probability'] > 1.0] = 1.0

In [21]:
test[['deal_probability']].to_csv('predictions/submission.csv')

In [23]:
import pickle
model_name = 'rf_overfit_ensembled'
pickle.dump(model, open('saved_models/{}.pickle'.format(model_name), 'wb'))
# model = pickle.load(open('saved_models/{}.pickle'.format(model_name), 'rb'))